# Agents 
An Agent in LangChain is a system that uses a Large Language Model as its "brain"<br><br>The core mechanism by which an agent operates is a continuous loop often referred to as the "Observation, Thought, Action" loop<br><br> Available tools via langchain_community, custom tools via @tool decorator

In [1]:
from langchain_core.tools import tool

@tool
def add_numbers(a: int, b: int) -> int:
    """Adds two numbers together."""
    return a + b
print(add_numbers.name)
print(add_numbers.description)

add_numbers
Adds two numbers together.


In [2]:
@tool
def get_current_weather(location: str) -> str:
    """Get the current weather in a given location.
    The location should be a city name, e.g., 'London' or 'New York'.
    Returns a string describing the weather.
    """
    if "london" in location.lower():
        return "It's cloudy with a chance of rain, 15°C."
    elif "new york" in location.lower():
        return "Sunny and warm, 25°C."
    else:
        return f"Weather data for {location} is not available."
get_current_weather.invoke("London")

"It's cloudy with a chance of rain, 15°C."

### BaseTool class for building async tools<br> _run(), _arun() methods,

In [21]:
from langchain_core.tools import BaseTool

class SimpleClaculator(BaseTool):
    name: str = 'SimpleCalculator'
    description: str = "Performs basic arithmetic operations. Input format: 'operation number1 number2' (e.g., 'add 5 10'). Supported operations: add, subtract, multiply,"
    def _run(self,query: str) -> str:
        """Use the calculator to perform a calculation."""
        parts = query.split()
        if len(parts) != 3:
            return "Please provide a valid calculation in the format: 'add 15 10."
        op, num1,num2 = parts[0], parts[1], parts[2]
        
        if op== "add":
            return str(float(num1) + float(num2))
        elif op == "subtract":
            return str(float(num1) - float(num2))
        elif op == "multiply":
            return str(float(num1) * float(num2))
    
    def _arun(self, query: str) -> str:
        """Use the calculator to perform a calculation asynchronously."""
        return self._run(query)
    
calci= SimpleClaculator()
calci.invoke("add 5 10")

'15.0'

### ReAct Agents<br> Resoning+Acting:-> Thought, Action, Observation
create_react_agent()

In [ ]:
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_react_agent

load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
chat_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)

tools = [calci, get_current_weather]

from langchain_core.prompts import PromptTemplate

# Simple string-based prompt template for ReAct
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

react_agent = create_react_agent(
    llm=chat_model,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor( #agent executer is the one who waits for agent to ask for tools and provide it
    agent=react_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

response = agent_executor.invoke({
    "input": "What is the weather in London? Also, add 5 and 10."
})
print(response['output'])



> Entering new AgentExecutor chain...
I need to first get the weather in London and then add 5 and 10. I will use the tools accordingly.
Action: get_current_weather
Action Input: LondonIt's cloudy with a chance of rain, 15°C.I now know the weather in London. I still need to add 5 and 10.
Action: SimpleCalculator
Action Input: add 5 1015.0I now have both the weather in London and the sum of 5 and 10.
Final Answer: The weather in London is cloudy with a chance of rain, 15°C. The sum of 5 and 10 is 15.0.

> Finished chain.
The weather in London is cloudy with a chance of rain, 15°C. The sum of 5 and 10 is 15.0.


# OpenAI function Agents
### How it works:
<ol>

<li>You define your tools (functions) and provide their schemas (name, description, parameters) to the LLM.</li>

<li>When you query the LLM, it decides if any of the provided tools are relevant to the user's request.</li>

<li>If a tool is relevant, the LLM generates a structured tool_calls message, specifying the tool's name and the arguments to call it with.</li>

<li>The Agent Executor then intercepts this tool_calls message, executes the actual Python function corresponding to the tool, and returns the result as an Observation.</li>
<li>This Observation is then fed back to the LLM for the next turn of reasoning.</li>

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
agent_llm= ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)

@tool
def get_stock_price(ticker: str) -> float:
    """Fetches the current stock price for a given company ticker symbol.
    Example: 'AAPL' for Apple, 'GOOG' for Google.
    Returns the price as a float.
    """
    # This is a dummy implementation. In a real app, you'd call a financial API.
    stock_prices = {"AAPL": 175.50, "GOOG": 150.20, "MSFT": 420.00, "AMZN": 180.10}
    price = stock_prices.get(ticker.upper())
    if price:
        return price
    else:
        return f"Stock price for {ticker.upper()} not found."
@tool
def calculate_compound_interest(principal: float, rate: float, time_years: int) -> float:
    """Calculates the compound interest.
    Args:
        principal (float): The initial amount of money.
        rate (float): The annual interest rate (as a decimal, e.g., 0.05 for 5%).
        time_years (int): The number of years the money is invested or borrowed for.
    Returns the final amount after compounding.
    """
    return principal * (1 + rate)**time_years
tools_fc=[get_stock_price, calculate_compound_interest]

prompt_fc = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful financial assistant use provided tools to answer the question."),
    ("human", "Question: {input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

tool_calling_agent = create_tool_calling_agent(
    llm=agent_llm,
    tools=tools_fc,
    prompt=prompt_fc
)

agent_executor_fc = AgentExecutor(
    agent=tool_calling_agent,
    tools=tools_fc,
    verbose=True, #prints every step for Debugging Agents thoughts
    handle_parsing_errors=True
)


In [26]:

response_1= agent_executor_fc.invoke({
    "input": "What is the current stock price of apple? Also, calculate the compound interest for a principal of 1000 at a rate of 5% for 10 years."
})
print(response_1['output'])

response_2 = agent_executor_fc.invoke({
    "input": "What is the current stock price of Microsoft? Also, calculate the compound interest for a principal of 2000 at a rate of 3% for 5 years."
})
print(response_2['output'])



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'ticker': 'AAPL'}`


175.5
Invoking: `calculate_compound_interest` with `{'rate': 0.05, 'principal': 1000.0, 'time_years': 10.0}`


1628.894626777442The current stock price of Apple is $175.5. The compound interest for a principal of 1000 at a rate of 5% for 10 years is $1628.89.

> Finished chain.
The current stock price of Apple is $175.5. The compound interest for a principal of 1000 at a rate of 5% for 10 years is $1628.89.


> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'ticker': 'MSFT'}`


420.0
Invoking: `calculate_compound_interest` with `{'time_years': 5.0, 'principal': 2000.0, 'rate': 0.03}`


2318.5481486000003The current stock price of Microsoft is $420. The compound interest for a principal of 2000 at a rate of 3% for 5 years is $2318.55.

> Finished chain.
The current stock price of Microsoft is $420. The compound interest for a principal of 2000 at a rate of 3% for 5 y

### Agent Memory with *RunnableWithMessageHistory()* for AgentExecuter
Wrapping the Agentexecutor inside the RunnableWithMessageHistory

In [38]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
chat_memory_agent=ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)

prompt_memory=ChatPromptTemplate.from_messages([
    ("system", "You are a helpful financial assistant use provided tools to answer the question."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "Question: {input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
tool_call_mem_agent = create_tool_calling_agent(
    llm=chat_memory_agent,
    tools=tools_fc,
    prompt=prompt_memory
)
agent_exec_with_mem=AgentExecutor(
    agent=tool_call_mem_agent,
    tools=tools_fc,
    verbose=True,
    handle_parsing_errors=True
)
store_mem = {}
def get_memory(session_id: str)->ChatMessageHistory:
    """Function to retrieve chat history for a given session."""
    if session_id not in store_mem:
        store_mem[session_id] = ChatMessageHistory()
    return store_mem[session_id]

    

agent_with_mem=RunnableWithMessageHistory(
    runnable=agent_exec_with_mem,
    get_session_history=get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

session_id="123"
prompt1="hi my name is alex, what is the current stock price of apple? Also, calculate the compound interest for a principal of 1000 at a rate of 5% for 10 years."
response_1= agent_with_mem.invoke(
    {"input": prompt1},
    config={"configurable":{"session_id": session_id}}
)
print(response_1['output'])

response_2= agent_with_mem.invoke(
    {"input": "what was my name again?"},
    config={"configurable":{"session_id": session_id}}
)
print(response_2['output'])



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'ticker': 'AAPL'}`


175.5
Invoking: `calculate_compound_interest` with `{'principal': 1000.0, 'rate': 0.05, 'time_years': 10.0}`


1628.894626777442Hi Alex, the current stock price of Apple is $175.5. The compound interest for a principal of 1000 at a rate of 5% for 10 years is $1628.89.

> Finished chain.
Hi Alex, the current stock price of Apple is $175.5. The compound interest for a principal of 1000 at a rate of 5% for 10 years is $1628.89.


> Entering new AgentExecutor chain...
Your name is Alex.

> Finished chain.
Your name is Alex.


## Human in a loop

In [41]:
@tool
def get_human_approval(action:str)->str:
    """Get approval from a human for senstive action."""
    print(f"Action requested: {action}")
    return "approved" if input("Do you approve this action? (yes/no): ").strip().lower()=="yes" else "Denied"

tools_with_human_approval = tools_fc + [get_human_approval]
prompt_with_human_approval = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful  assistant use provided tools to answer the question and perform actions, checking stock rpice is a sensitive action that requires human approval."),
    ("human", "Question: {input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

huma_loop_agent = create_tool_calling_agent(
    llm=chat_memory_agent,
    tools=tools_with_human_approval,
    prompt=prompt_with_human_approval
)
agent_exec_with_human_approval = AgentExecutor(
    agent=huma_loop_agent,
    tools=tools_with_human_approval,
    verbose=True,
    handle_parsing_errors=True
)

response=agent_exec_with_human_approval.invoke({
    "input": "get me the apple stock price"
})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `get_human_approval` with `{'action': 'get the apple stock price'}`


Action requested: get the apple stock price
approved
Invoking: `get_stock_price` with `{'ticker': 'AAPL'}`


175.5The current stock price for Apple (AAPL) is $175.5.

> Finished chain.
The current stock price for Apple (AAPL) is $175.5.
